In [5]:
!cat .env

STAGE=PRODUCTION


In [6]:
import json
from tqdm.notebook import tqdm
from openpyxl import load_workbook
from django.contrib.gis.geos import Point
from helpers.signals import DisableSignals
from helpers.strings import get_autonumber_array
from simple_history.utils import bulk_create_with_history
from uuid import uuid4
from django.utils import timezone
import random
from datetime import timedelta
from datetime import datetime
from collections import defaultdict

In [7]:
arteriscs = User.objects.get(username="arteriscs")

In [8]:
company = Company.objects.get(name="Arteris Via Paulista - SISOAE")

In [9]:
def get_sheet_values(filename, sheet_index):
    wb = load_workbook(filename + ".xlsx")
    sheetname = wb.sheetnames[sheet_index]
    ws = wb[sheetname]
    
    header = []
    values = []

    for index, row in enumerate(ws.rows):
        if index == 0:
            header = list([a.value for a in row])
            continue
        obj = {}
        for col_index, cell in enumerate(row):
            value = cell.value
            obj[header[col_index]] = value
        values.append(obj)
        
    return values

In [10]:
rotineira2023 = get_sheet_values('inspecoes_v1', 0)

In [11]:
prefix_blacklist = ["1) ","2) ","3) ","4) ","5) ","6) ","7) ","8) ","9) ","10) ","11) ","12) ","13) ","14) ","15) ",
                    "1. ","2. ","3. ","4. ","5. ","6. ","7. ","8. ","9. ","10. ","11. ","12. ","13. ","14. ","15. ",
                    "1.","2.","3.","4.","5.","6.","7.","8.","9.","10.","11.","12.","13.","14.","15.",
                    "1 – ","2 – ","3 – ","4 – ","5 – ","6 – ","7 – ","8 – ","9 – ","10 – ","11 – ","12 – ","13 – ","14 – ","15 – ",
                    "1 - ","2 - ","3 - ","4 - ","5 - ","6 - ","7 - ","8 - ","9 - ","10 - ","11 - ","12 - ", "13 - ","14 - ","15 - ",
                    "10 – ", "9  – ", "1- ",  "2 – ","4 –", "5– ", "6  - ", "3  – ", "8  – ", "5  – ", "4  – ", "6  – ", "-1. ",
                   "• ", "•"]

In [12]:
def separate_photos(value):
    value=tuple(value.split(' '))
    
    return value

In [13]:
def parse_terapias(terapias):
    terapias_list = ([
        b.strip(' .,;') for b in terapias.split('\n') 
        if b.strip(' .,;') 
        and b != "Por ora não há recomendações de terapia." 
        and b != "-"
        and b != "- Indicações de Terapia"]
        if terapias else [])
    ret_terapias = []
#     print(terapias_list)

    
    for terapia in terapias_list:
#         print(terapia)
        if terapia == '':
            continue
        if terapia[0:2] in prefix_blacklist:
            terapia = terapia[2:]
        elif terapia[0:3] in prefix_blacklist:
            terapia = terapia[3:]            
        elif terapia[0:4] in prefix_blacklist:
            terapia = terapia[4:]
        elif terapia[0:5] in prefix_blacklist:
            terapia = terapia[5:]

        if terapia == "Implantação de sinalização horizontall":
            terapia = "Implantação de sinalização horizontal"
            
        if terapia == "- Indicações de Terapia":
            continue
#         print(terapia)
        

        
        terapia=terapia.replace('foto','Foto').replace(').', ')').replace('.', ';').replace(' (', '; (').replace('.(', '; (').replace('. (', '; (').replace(';;', ';').replace(';(', '; (').replace('  ', ' ')
#         print(terapia)
#         if 'e' in terapia.split('; ')[1]:
#             print(terapia.split('; ')[1].strip('.,;)(').replace(',', '').replace(' e', '').replace(' 1', ' 01').replace(' 2', ' 02').replace(' 3', ' 03').replace(' 4', ' 04').replace(' 5', ' 05').replace(' 6', ' 06').replace(' 7', ' 07').replace(' 8', ' 08').replace(' 9', ' 09').replace('010', '10').replace('011', '11').replace('012', '12').replace('013', '13').replace('014', '14').replace('015', '15').replace('016', '16').replace('017', '17').replace('018', '18').replace('019', '19'))
#         terapia = (terapia.split('; ')[0].strip(' .,;'), separate_photos(terapia.split('; ')[1].strip(' .,;)(').replace(',', '').replace(' e', '').replace(' 1', ' 01').replace(' 2', ' 02').replace(' 3', ' 03').replace(' 4', ' 04').replace(' 5', ' 05').replace(' 6', ' 06').replace(' 7', ' 07').replace(' 8', ' 08').replace(' 9', ' 09').replace('010', '10').replace('011', '11').replace('012', '12').replace('013', '13').replace('014', '14').replace('015', '15').replace('016', '16').replace('017', '17').replace('018', '18').replace('019', '19').replace('020', '20').replace('021', '21').replace('022', '22').replace('Fotos', '').replace('Foto', '').replace('023', '23').replace('024', '24').replace('025', '25').replace('026', '26').replace('  ', ' ').strip()))
        fotos = separate_photos(terapia.split('; ')[1].strip(' .,;)(').replace(',', '').replace(' e', '').replace(' 01', ' 1').replace(' 02', ' 2').replace(' 03', ' 3').replace(' 04', ' 4').replace(' 05', ' 5').replace(' 06', ' 6').replace(' 07', ' 7').replace(' 08', ' 8').replace(' 09', ' 9').replace('010', '10').replace('011', '11').replace('012', '12').replace('013', '13').replace('014', '14').replace('015', '15').replace('016', '16').replace('017', '17').replace('018', '18').replace('019', '19').replace('020', '20').replace('021', '21').replace('022', '22').replace('Fotos', '').replace('Foto', '').replace('023', '23').replace('024', '24').replace('025', '25').replace('026', '26').replace('  ', ' ').strip()) if len(terapia.split('; ')) > 1 else ''
        # Cria um range da foto inicial até a foto final na tupla fotos
        if 'a' in fotos:
            dados=list(fotos)
            dados.remove('a')
            first=int(dados[0])
            last=int(dados[-1])
            new_lista=[]
            for b in range(first,last+1):
                new_lista.append(b)
            new_lista = tuple(new_lista)
            fotos=new_lista
        terapia = (terapia.split('; ')[0].strip(' .,;'), fotos if len(terapia.split('; ')) > 1 and fotos else '')
#         terapia[1] = separate_photos(terapia[1])
        ret_terapias.append(terapia)

        
        
#     print(terapias)
#     print(ret_terapias)
#     print('\n')
#     print('\n')
    return ret_terapias

In [14]:
all_terapias = []

In [15]:
for i,a in enumerate(tqdm(rotineira2023)):
    try:
        all_terapias += parse_terapias(a['Terapia 1'])
    except Exception as e:
        print(e, i+2,a['Terapia 1'])

  0%|          | 0/416 [00:00<?, ?it/s]

In [16]:
list(a for a in all_terapias if a[0] == 'Tratar fissura no pavimento asfáltico')

[('Tratar fissura no pavimento asfáltico', ('12',)),
 ('Tratar fissura no pavimento asfáltico', ('7',)),
 ('Tratar fissura no pavimento asfáltico', ('12', '13'))]

In [17]:
all_terapias[500]

('Tratar armaduras expostas na testeira', ('9', '10'))

In [18]:
def get_connected_reference(company, resource, key, value, default=None):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    try:
        return next(a['value'] for a in select_options if a['name'].lower() == value.lower())
    except StopIteration:
        return default

In [19]:
def get_form_reference(form, key, value):
    field = next(a for a in form.form_fields['fields'] if a['apiName'] == key)
    try:
        return next(a['value'] for a in field['selectOptions']['options'] if a['name'].lower() == value.lower())
    except StopIteration:
        print("DEURUIMM")
        print(form, key, value)
        return ""

In [20]:
def get_job_number(company):

    instance_type = "job"
    key_name = "{}_name_format".format(instance_type)
    # Get datetime and serial arrays
    data = get_autonumber_array(company, instance_type)
    # Get company prefix
    if "company_prefix" in company.metadata:
        data["prefixo"] = company.metadata["company_prefix"]
    else:
        data["prefixo"] = "[{}]".format(company.name)
    # Make number
    try:
        if key_name in company.metadata:
            number = company.metadata[key_name].format(**data)
        else:
            raise Exception("Variáveis de nome inválidas!")
    except Exception as e:
        print(e)
        # Fallback
        # UHIT-job-2018.0001
        number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)

    return number

In [21]:
def get_reporting_number(company, occurrence_kind):

    key_name = "RP_name_format"
    number_format = ""

    if key_name in company.metadata:
        try:
            number_format = company.metadata[key_name][
                occurrence_kind
            ]
        except Exception:
            if "default" in company.metadata[key_name]:
                number_format = company.metadata[key_name][
                    "default"
                ]
            else:
                raise serializers.ValidationError(
                    "Variáveis de nome inválidas!"
                )
    else:
        raise serializers.ValidationError("Variáveis de nome inválidas!")

    instance_type = number_format["type"]

    # Get datetime and serial arrays
    data = get_autonumber_array(company.uuid, instance_type)

    # Get company prefix
    if "company_prefix" in company.metadata:
        data["prefixo"] = company.metadata["company_prefix"]
    else:
        data["prefixo"] = "[{}]".format(company.name)

    # Make number
    try:
        number = number_format["format"].format(**data)
    except Exception as e:
        print(e)
        # Fallback
        # UHIT-RG-2018.0001
        number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)

    return number

In [22]:
def get_direction_name(company, value):
    choices=company.custom_options['reporting']['fields']['direction']['select_options']['options']
#     items={a['name']: a['short'] for a in choices}
    items={a['value']: a['short'] for a in choices}
    
    return items[value].replace('/','')

In [23]:
def get_direction_value(company, name):
    choices=company.custom_options['reporting']['fields']['direction']['select_options']['options']
#     items={a['name']: a['short'] for a in choices}
    items={a['short'].replace('/',''): a['value'] for a in choices}
    
    return items[name.upper()]

In [24]:
executado = ServiceOrderActionStatus.objects.get(companies=company, name="Executado")
identificado = ServiceOrderActionStatus.objects.get(companies=company, name="Identificado")
firm = Firm.objects.get(company=company, name="Não atribuído")
form_inspecao_rotineira = OccurrenceType.objects.get(company=company, name="Inspeção Rotineira")

In [25]:
oae=Reporting.objects.get(company=company, form_data__artesp_service='05.03.29.01.00.18',
                                    occurrence_type__occurrence_kind="2")
inspection_2023 = oae.children.filter(executed_at__year=2023, occurrence_type__name__icontains="rotineira")
inspection_2023

<QuerySet [<Reporting: [Arteris Via Paulista - SISOAE] INS-2024.03450 - 2023-07-01 03:00:00+00:00>]>

In [34]:
rotineira2023[0]

{'Nome do documento': 'FICHA DE INSPEÇÃO',
 'km': '002+900',
 'km final': 0,
 'km de Projeto': 0,
 'km final de Projeto': 0,
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'cs',
 'Encontrado em': datetime.datetime(2023, 7, 1, 0, 0),
 'Executado em': None,
 'Prazo': None,
 'Sentido': 'Ambos',
 'Classe': 'Inspeção Rotineira de OAE',
 'Faixa': 'Não se Aplica',
 'Rodovia': 'SP-255',
 'Inspeção inicial': datetime.datetime(2019, 10, 1, 0, 0),
 'Última inspeção rotineira': datetime.datetime(2022, 7, 1, 0, 0),
 'Inspeção especial': '-',
 'Reparos': 'Pintura de proteção nos elementos da OAE e tratamento das fissuras nas juntas de dilatação.',
 'Reformas': 'Não há indícios.',
 'Reforços': 'Não há indícios.',
 'Item de serviço ARTESP': '05.03.29.01.00.01',
 'Tipo de obra': 'Passagem Superior',
 'Outro tipo de obra': 'Não sei o que colocar aqui',
 'Característica visual do tabuleiro': 'Em bom estado.',
 'Característica visual das juntas de dilatação': 'As juntas de dilatação encontram-se encobe

In [58]:
objects = []
terapia_objects = []
jobs = []
photos=[]
photos_general=[]
pdf_general=[]
counter_inside=0

for a in tqdm(rotineira2023):
    
#     km = float(a['km'].replace("+", "."))
    road_name = a['Rodovia'].replace(' ', '-').replace('/','-')
    road_name_photo = a['Rodovia'].replace(' ', '').replace('-','').replace('/','-')
#     print(road_name_photo)
    
    try:
        oae = Reporting.objects.get(company=company, form_data__artesp_service=a['Item de serviço ARTESP'],
                                    occurrence_type__occurrence_kind="2")
    except Exception as e:
        print(f"OAE não encontrada - Item da ARTESP {a['Item de serviço ARTESP']}")
        continue
        
    inspection_2023 = oae.children.filter(executed_at__year=2023, occurrence_type__name__icontains="rotineira")
    if inspection_2023.exists():
        print(f'Inspeção já existe - {inspection_2023.first().number}')
#         inspection_2023.first().delete()
        continue

    inspection_uuid = uuid4()   
    
    photos_uuid=defaultdict(list)
    
    terapias = parse_terapias(a['Terapia 1'])
#     print(terapias)
    extensions=['.jpg','.jpeg', '.png']
    
    for terapia in terapias:
        
        for photo_name in terapia[1]:
            if photo_name == "":
                continue
            counter_inside+=1
            for i, item in enumerate(extensions):
#                 print(photo_name,item)
        
                photo=ReportingFile(upload='sisoae_2023/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/Fotos/' + 'Foto' + str(photo_name) + item,
                                uuid=uuid4(), description='Imagem da Terapia', reporting_id=inspection_uuid, created_by=arteriscs)
#                 photo_general=ReportingFile(upload='sisoae_2023/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/Fotos/' + 'Foto' + str(photo_name) + item,
#                                 uuid=uuid4(), description='Imagem da Terapia',  kind='antes', reporting_id=inspection_uuid, created_by=arteriscs)
#                 pdf_exists = ReportingFile.objects.filter(upload='sisoae_2023/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction)+'.pdf',
#                                 reporting_id=inspection_uuid, created_by=arteriscs)
#                 if not pdf_exists.exists():
#                     pdf=ReportingFile(upload='sisoae_2023/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction)+'.pdf',
#                                 uuid=uuid4(), description='Arquivo da Terapia',  kind='antes', reporting_id=inspection_uuid, created_by=arteriscs)
                try:
                    photo.upload.size
#                     photo_general.upload.size
#                     pdf_general.upload.size
                except Exception as e:
                    if i == 2:
                        print('sisoae_2023/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/Fotos/' + 'Foto' + str(photo_name) + item)
                    continue
                else:
                    photos_uuid[terapia[0]].append(str(photo.uuid))
                    photos.append(photo)
#                     photos_general.append(photo_general)
#                     if not pdf_exists.exists():
#                         pdf_general.append(pdf)
                    break
#     continue
     
#     print(photos_uuid)
    terapias = [
        {"description": terapia, "occurrence_type":"f7858877-f5a5-4416-ba1a-39e5f320b30d", "treatment_images": photo_uuid}
        for terapia, photo_uuid in photos_uuid.items() if terapia is not None and terapia != []
    ]
#     print(terapias)
    
    try:
        structural_classification = get_form_reference(form_inspecao_rotineira, 'structuralClassification', a['Estrutural'].strip()) if not "-" in a['Estrutural'] else ""
        functional_classification = get_form_reference(form_inspecao_rotineira, 'functionalClassification', a['Funcional'].strip()) if not "-" in a['Funcional'] else ""
        wear_classification = get_form_reference(form_inspecao_rotineira, 'wearClassification', a['Durabilidade'].strip()) if not "-" in a['Durabilidade'] else ""
    except Exception as e:
        print(f'Erro na função "get_form_reference" - {e}')
        structural_classification = ""
        functional_classification = ""
        wear_classification = ""
        
    if isinstance(a['Encontrado em'], str):
        a['Encontrado em'] = datetime.strptime(a['Encontrado em'], "%d/%m/%Y")
        
    try:
        obj = {
            "uuid": inspection_uuid,
            "number": get_reporting_number(company, form_inspecao_rotineira.occurrence_kind),
            "km": oae.km,
            "end_km": oae.end_km,
            'direction': oae.direction,
            'lane': oae.lane,
            'occurrence_type': form_inspecao_rotineira,
            'status': executado,
            'executed_at': a['Encontrado em'],
            'found_at': a['Encontrado em'],
            'created_at': a['Encontrado em'],
            'road_name': oae.road_name,
            'parent_id': oae.uuid,
            'form_data': {
                'repairs': a['Reparos'].replace('\n', '') if a['Reparos'] is not None else None,
                'reform': a['Reformas'].replace('\n', '') if a['Reformas'] is not None else None,
                'intervention': a['Reforços'].replace('\n', '') if a['Reforços'] is not None else None,
                'visual_structure_board': a['Característica visual do tabuleiro'].replace('\n', '')if a['Característica visual do tabuleiro'] is not None else None,
                'visual_structure_joints': a['Característica visual das juntas de dilatação'].replace('\n', '') if a['Característica visual das juntas de dilatação'] is not None else None, 
                'visual_structure_supports': a['Característica visual dos aparelhos de apoio'].replace('\n', '') if a['Característica visual dos aparelhos de apoio'] is not None else None,
                'visual_structure_walls': a['Característica visual dos pilares'].replace('\n', '') if a['Característica visual dos pilares'] is not None else None,
                'visual_structure_connections': a['Característica visual dos encontros'].replace('\n', '') if a['Característica visual dos encontros'] is not None else None,
                'paving_conditions': a['Pavimento'].replace('\n', '') if a['Pavimento'] is not None else None,
                'coasting': a['Acostamento'].replace('\n', '') if a['Acostamento'] is not None else None,
                'drains': a['Drenagem'].replace('\n', '') if a['Drenagem'] is not None else None,
                'guard_rail': a['Guarda-Corpos'].replace('\n', '') if a['Guarda-Corpos'] is not None else None,
                'defenses': a['Defensas rígidas'].replace('\n', '') if a['Defensas rígidas'] is not None else None,
                'slope': a['Taludes'].replace('\n', '') if a['Taludes'] is not None else None,
                'lightning': a['Iluminação'].replace('\n', '') if a['Iluminação'] is not None else None,
                'signaling': a['Sinalização'].replace('\n', '') if a['Sinalização'] is not None else None,
                'templates': a['Gabaritos'].replace('\n', '') if a['Gabaritos'] is not None else None,
                'dolfins': a['Dolfins'].replace('\n', '') if a['Dolfins'] is not None else None,
                'additional_info': a['Informações complementares'].replace('\n', '') if a['Informações complementares'] is not None else None,
                "construction": oae.form_data['construction'] if 'construction' in oae.form_data and oae.form_data['construction'] is not None else None,
                'therapy': terapias if terapias and terapias != [] else None,
                'structural_classification': structural_classification,
                'functional_classification': functional_classification, 
                'wear_classification': wear_classification,
                "artesp_service": a["Item de serviço ARTESP"],
            },
            'created_by': arteriscs,
            'point': oae.point,
            'road': oae.road,
            'company': oae.company,
            'firm': firm
        }
        try:
            if not obj['form_data']['therapy']:
                del obj['form_data']['therapy']
            inspection = Reporting(**obj)
            objects.append(inspection)
        except Exception as e:
            print(f'Erro de criação de terapia do item {a["Item de serviço ARTESP"]} - {e}')
            continue
    except AttributeError as e:
        print(f'Erro geral - {e}')
        continue
                  
    continue
        
#     job_title = "Fase 2 - {}".format(oae.form_data['artesp_service'])
#     job_color = "#7030A0"
#     job_exists = Job.objects.filter(title=job_title, company=company).exists()
    
#     if job_exists:
#         job_title = "Fase 3 - {}".format(oae.form_data['artesp_service'])
#         job_color = "#00CCFF"
        
#     job = Job(
#         title=job_title,
#         description=job_color,
#         number=get_job_number(company),
#         start_date=a['Encontrado em'],
#         end_date=a['Encontrado em'] + timedelta(days=365),
#         inspection=inspection,
#         parent_inventory=oae,
#         created_by=arteriscs,
#         firm=firm,
#         worker=arteriscs,
#         company=company
#     )
#     jobs.append(job)
    
#     for terapia in terapias:
#         form_terapia = OccurrenceType.objects.get(company=company, uuid=terapia['occurrence_type'])
#         obj = {
#             "number": get_reporting_number(company, form_terapia.occurrence_kind),
#             "km": oae.km,
#             "end_km": oae.end_km,
#             'direction': oae.direction,
#             'lane': oae.lane,
#             'occurrence_type': form_terapia,
#             'status': identificado,
#             'found_at': a['Encontrado em'],
#             'created_at': a['Encontrado em'],
#             'road_name': oae.road_name,
#             'parent_id': oae.uuid,
#             'form_data': {
#                 'description': terapia['description']
#             },
#             'created_by': arteriscs,
#             'point': oae.point,
#             'road': oae.road,
#             'company': oae.company,
#             'firm': firm,
#             'job': job
#         }
#         try:
#             terapia_obj = Reporting(**obj)
#             terapia_objects.append(terapia_obj)
#         except Exception:
#             print('terapia error')

  0%|          | 0/416 [00:00<?, ?it/s]

sisoae_2023/SP255/SP255-km037+318-N/Fotos/Foto10.png
sisoae_2023/SP255/SP255-km037+318-N/Fotos/Foto11.png
sisoae_2023/SP255/SP255-km037+318-N/Fotos/Foto12.png
sisoae_2023/SP255/SP255-km084+950-NS/Fotos/Foto12.png
sisoae_2023/SP255/SP255-km084+950-NS/Fotos/Foto10.png
sisoae_2023/SP255/SP255-km084+950-NS/Fotos/Foto11.png
sisoae_2023/SP255/SP255-km084+950-NS/Fotos/Foto9.png
sisoae_2023/SP255/SP255-km124+000-NS/Fotos/FotoNão.png
sisoae_2023/SP255/SP255-km124+000-NS/Fotos/Fotovisualizado.png
sisoae_2023/SP330/SP330-km274+300-S/Fotos/Foto20.png
sisoae_2023/SP330/SP330-km306+900-T/Fotos/Foto12.png
sisoae_2023/SP334/SP334-km375+800-T/Fotos/Foto12.png
sisoae_2023/SP334/SP334-km375+800-T/Fotos/Foto11.png
sisoae_2023/SP334/SP334-km383+200-NS/Fotos/FotoS.png
sisoae_2023/SP334/SP334-km394+400-NS/Fotos/Foto12.png
OAE não encontrada - Item da ARTESP 03.03.04.90
sisoae_2023/SP334/SP334-km403+970-NS/Fotos/Foto16.png
sisoae_2023/SP345/SP345-km026+660-L/Fotos/Foto11.png
sisoae_2023/SP345/SP345-km026+660-

In [59]:
len(photos)

1263

In [60]:
result1 = bulk_create_with_history(objects, Reporting, default_user=arteriscs)
result2 = bulk_create_with_history(photos, ReportingFile, default_user=arteriscs)
# result3 = bulk_create_with_history(photos_general, ReportingFile, default_user=arteriscs)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-07-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2023-07-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-11-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeFiel

In [64]:
reportings=Reporting.objects.filter(company=company, occurrence_type=form_inspecao_rotineira, created_at__gte='2024-2-20').order_by('number')
reportings.count(),reportings[0]

(415,
 <Reporting: [Arteris Via Paulista - SISOAE] INS-2024.03377 - 2023-07-01 03:00:00+00:00>)

In [63]:
for a in tqdm(reportings):
    if a.form_data.get('therapy') is None:
        print(a.uuid)
        del a.form_data['therapy']
        a.save()

  0%|          | 0/415 [00:00<?, ?it/s]

823ce451-4656-4d27-b604-56eabfb613f9


KeyError: 'therapy'

In [5]:
file_name='Lista de Fotos - SISOAE 2022'

with open(file_name + '.txt', 'r+') as fo:
    photo_list=json.load(fo)

In [25]:
photos=[]

for key, values in tqdm(photo_list.items()):
    
    try:
        reporting=Reporting.objects.get(company=company, occurrence_type=form_inspecao_rotineira, created_at__gte='2022-12-10', created_by=arteriscs, road_name=key.split('*')[0].replace('-', '/').replace(' ', '-'), km=key.split('*')[1].replace('+', '.'), direction=get_direction_value(company, key.split('*')[2]))
        for value in values:
            photo=ReportingFile(upload='sisoae_2022_vpt/' + key.split('*')[0] + '/' + key.split('*')[1] + ' ' + key.split('*')[2] + '/' + value,
                            uuid=uuid4(), description='Imagem da Inspeção', kind='antes', reporting_id=reporting.uuid, created_by=arteriscs)
            
            try:
                photo.upload.size
                photos.append(photo)
            except Exception as e:
                print(key, value)
    except Exception as e:
        print(key)
        pass

  0%|          | 0/411 [00:00<?, ?it/s]

/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2021-11-26 00:00:00) while time zone support is active.
  RuntimeWarning)


SP 255*178+080*T
SP 255*179+020*T


In [26]:
# photo_result = bulk_create_with_history(photos, ReportingFile, default_user=arteriscs)

In [2]:
from RoadLabsAPI.settings import credentials
import boto3



# session = boto3.client(
#     "s3",
#     aws_access_key_id=credentials.AWS_ACCESS_KEY_ID,
#     aws_secret_access_key=credentials.AWS_SECRET_ACCESS_KEY,
# )

s3 = boto3.resource('s3')

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [3]:
pdf_list={}

In [37]:
my_bucket = s3.Bucket('spotway-roadlabs-assets')
for a in my_bucket.objects.filter(Prefix="media/private/sisoae_2023/pdf"):
    print(a.key)

media/private/sisoae_2023/pdf/SP255-km002+900-T.pdf
media/private/sisoae_2023/pdf/SP255-km002+920-T.pdf
media/private/sisoae_2023/pdf/SP255-km006+700-NS.pdf
media/private/sisoae_2023/pdf/SP255-km006+900-T.pdf
media/private/sisoae_2023/pdf/SP255-km009+000-NS.pdf
media/private/sisoae_2023/pdf/SP255-km009+250-NS.pdf
media/private/sisoae_2023/pdf/SP255-km009+500-T.pdf
media/private/sisoae_2023/pdf/SP255-km011+300-N.pdf
media/private/sisoae_2023/pdf/SP255-km011+300-S.pdf
media/private/sisoae_2023/pdf/SP255-km013+620-T.pdf
media/private/sisoae_2023/pdf/SP255-km017+500-NS.pdf
media/private/sisoae_2023/pdf/SP255-km017+944-N.pdf
media/private/sisoae_2023/pdf/SP255-km017+944-S.pdf
media/private/sisoae_2023/pdf/SP255-km019+000-NS.pdf
media/private/sisoae_2023/pdf/SP255-km020+400-NS.pdf
media/private/sisoae_2023/pdf/SP255-km022+900-T.pdf
media/private/sisoae_2023/pdf/SP255-km024+250-N.pdf
media/private/sisoae_2023/pdf/SP255-km024+250-S.pdf
media/private/sisoae_2023/pdf/SP255-km027+100-T.pdf
media/

In [30]:
my_bucket = s3.Bucket('spotway-roadlabs-assets')

for my_bucket_object in my_bucket.objects.filter(Prefix="media/private/sisoae_2023/pdf"):
    if '.pdf' in my_bucket_object.key:
        for a in rotineira2023:
            if '05.03.29.20.00.02' in  a['Item de serviço ARTESP']:
                print(a['Item de serviço ARTESP'])
            try:
                oae = Reporting.objects.get(company=company, form_data__artesp_service=a['Item de serviço ARTESP'],
                                            occurrence_type__occurrence_kind="2")
            except Exception as e:
                print(f"OAE não encontrada - Item da ARTESP {a['Item de serviço ARTESP']}")
                continue
            else:
                inspection_2023 = oae.children.filter(created_at='2024-2-20', occurrence_type__name__icontains="rotineira")
                if inspection_2023.exists():
                    if my_bucket_object.key.split('/')[-1] not in pdf_list.values():
                        pdf_list[inspection_2023.first().number] = my_bucket_object.key.split('/')[-1]

OAE não encontrada - Item da ARTESP 03.03.04.90
05.03.29.20.00.02
OAE não encontrada - Item da ARTESP 03.03.04.90
05.03.29.20.00.02


KeyboardInterrupt: 

In [31]:
pdf_list

{}

# PDF Upload

In [33]:
oaes=Reporting.objects.filter(company=company, occurrence_type__occurrence_kind="2")
oaes.count()

420

In [19]:
inspection_2022.road_name.replace('-','').replace('/','-')

NameError: name 'inspection_2022' is not defined

In [34]:
pdf_upload=[]

for oae in tqdm(oaes):
    
    try:
        inspection_2022 = oae.children.filter(executed_at__year=2023, created_at__year=2024, occurrence_type__name__icontains="rotineira").first()
        if inspection_2022:
            km="{:07.3f}".format(inspection_2022.km).replace(".", "+")
            rep_file = ReportingFile.objects.filter(upload='sisoae_2023/pdf/' + inspection_2022.road_name.replace('-','').replace('/','-') + '-km'+ km + "-" + get_direction_name(company, oae.direction) + ".pdf",
                                description='PDF', reporting_id=inspection_2022.uuid, created_by=arteriscs)
            if rep_file.exists():
                continue
            pdf_file=ReportingFile(upload='sisoae_2023/pdf/' + inspection_2022.road_name.replace('-','').replace('/','-') + '-km'+ km + "-" + get_direction_name(company, oae.direction) + ".pdf",
                                uuid=uuid4(), description='PDF', reporting_id=inspection_2022.uuid, created_by=arteriscs)
            pdf_file.upload.size
            pdf_upload.append(pdf_file)    
    except Exception as e:
        if inspection_2022:
            print(f"{inspection_2022.road_name.replace('-','').replace('/','-')}-km{km}-{get_direction_name(company, oae.direction)}")
            print(oae.form_data['artesp_service'])
            pass

  0%|          | 0/420 [00:00<?, ?it/s]

SP255-km037+318-N
05.03.01.20
SP255-km037+318-S
05.03.01.21
SP255-km084+950-NS
05.03.29.01.00.06
SP334-km320+400-N
05.03.04.05
SP334-km338+280-T
05.03.04.34
SPA214-255-km000+200-NS
05.03.29.13.00.01
SP345-km026+660-O
05.03.05.11
SP345-km026+660-L
05.03.05.10
SP334-km338+540-T
05.03.04.38
SP334-km361+000-S
05.03.04.64
SP330-km307+620-T
05.03.03.76.04
SP330-km307+690-O
05.03.03.76.05


In [35]:
len(pdf_upload)

401

In [36]:
result = bulk_create_with_history(pdf_upload, ReportingFile, default_user=arteriscs)

# Programações Criação

In [47]:
progs=get_sheet_values('Geral OAE 2024', 0)
progs[0]

{None: None,
 'ITEM INVENTÁRIO': '05.03.29.13.00.01',
 'NOME ESTRUTURA ARTESP': 'Galeria',
 'TIPO OAE': 'GALERIA',
 'RODOVIA': 'SPA 214/255',
 'KM': '000+200',
 'SENTIDO': 'N/S',
 'TRECHO': 'EXIST.',
 'LOTE': 'LOTE 4',
 'INICIAL PROGRAMADO RECUPERAÇÃO 2024': datetime.datetime(2025, 6, 1, 0, 0),
 'FINAL PROGRAMADO RECUPERAÇÃO 2024': datetime.datetime(2025, 7, 13, 0, 0),
 'TIPO DE OBRA RECUPERAÇÃO 2024': 'Recuperação',
 'TERAPIAS ': '1. Executar limpeza através de hidrojateamento em todos os elementos de concreto da OAE;\n2. Tratar a fissura no pavimento asfáltico do acostamento;\n3. Recompor áreas com armaduras expostas e segregações de concreto nas testeiras, na laje, nos encontros e nos muros ala através de reconstituição com argamassa polimérica.\n4. Retirar as pontas de ferro remanescentes da execução e tratar as áreas com argamassa polimérica;\n5. Tratar fissuras presentes nas paredes dos encontros através de selagem com argamassa polimérica;\n6. Executar juntas com delimitador de 

In [82]:
parse_terapias(progs[12]['TERAPIAS '])
# progs[14]['TERAPIAS '],progs[14]['FASE'],progs[14]['ITEM INVENTÁRIO']

[('Tratar manchas de eflorescência na face inferior da laje', ('9', '10')),
 ('Tratar fissuras no piso da ciclovia', ('11', '12'))]

In [76]:
for prog in tqdm(progs):
    oae = Reporting.objects.filter(occurrence_type__occurrence_kind='2',company=company,
                                  form_data__artesp_service=prog['ITEM INVENTÁRIO'].strip()).first()
    inspection = Reporting.objects.filter(occurrence_type=form_inspecao_rotineira,
                                       company=company,
                                      form_data__artesp_service=prog['ITEM INVENTÁRIO'].strip(),
                                         created_at__year=2024).first()
    if oae == None or inspection == None:
        print(oae,inspection)

  0%|          | 0/419 [00:00<?, ?it/s]

[Arteris Via Paulista - SISOAE] 05.03.29.01.00.63 - 2024-02-01 18:54:23.554000+00:00 None
[Arteris Via Paulista - SISOAE] 05.03.29.06.00.04 - 2021-04-15 03:00:00+00:00 None
[Arteris Via Paulista - SISOAE] 05.03.29.04.00.07 - 2024-02-01 19:46:58.558000+00:00 None
[Arteris Via Paulista - SISOAE] 05.03.04.81 - 2021-04-15 03:00:00+00:00 None
[Arteris Via Paulista - SISOAE] 05.03.04.90 - 2021-04-15 03:00:00+00:00 None


In [88]:
terapia_objects=[]
jobs=[]
for prog in tqdm(progs):
    oae = Reporting.objects.filter(occurrence_type__occurrence_kind='2',company=company,
                                  form_data__artesp_service=prog['ITEM INVENTÁRIO'].strip()).first()
    inspection = Reporting.objects.filter(occurrence_type=form_inspecao_rotineira,
                                       company=company,
                                      form_data__artesp_service=prog['ITEM INVENTÁRIO'].strip(),
                                         created_at__year=2024).first()
    if oae == None or inspection == None:
        continue
    job_title = "Fase {} - {}".format(prog['FASE'],oae.form_data['artesp_service'])
    job_exists = Job.objects.filter(title=job_title, company=company).exists()
    
    if job_exists:
        continue
        
    job = Job(
        title=job_title,
        description='#5566bc',
        number=get_job_number(company),
        start_date=prog['INICIAL PROGRAMADO RECUPERAÇÃO 2024'] if prog.get('INICIAL PROGRAMADO RECUPERAÇÃO 2024') else datetime.now(),
        end_date=prog['FINAL PROGRAMADO RECUPERAÇÃO 2024'] if prog.get('FINAL PROGRAMADO RECUPERAÇÃO 2024') else datetime.now() + timedelta(days=365),
        inspection=inspection,
        parent_inventory=oae,
        created_by=arteriscs,
        firm=firm,
        worker=arteriscs,
        company=company
    )
    jobs.append(job)
    if prog.get('TERAPIAS ') is not None and '-' not in prog.get('TERAPIAS '):
        terapias = parse_terapias(prog.get('TERAPIAS '))
        for terapia,fotos in terapias:
            form_terapia = OccurrenceType.objects.get(company=company, uuid='f7858877-f5a5-4416-ba1a-39e5f320b30d')
            obj = {
                "number": get_reporting_number(company, form_terapia.occurrence_kind),
                "km": oae.km,
                "end_km": oae.end_km,
                'direction': oae.direction,
                'lane': oae.lane,
                'occurrence_type': form_terapia,
                'status': identificado,
                'found_at': datetime.now(),
                'created_at': datetime.now(),
                'road_name': oae.road_name,
                'parent_id': oae.uuid,
                'form_data': {
                    'description': terapia
                },
                'created_by': arteriscs,
                'point': oae.point,
                'road': oae.road,
                'company': oae.company,
                'firm': firm,
                'job': job
            }
            try:
                terapia_obj = Reporting(**obj)
                terapia_objects.append(terapia_obj)
            except Exception:
                print('terapia error')

  0%|          | 0/419 [00:00<?, ?it/s]

In [92]:
terapia_objects[3].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fe1ec6c54d0>,
 'uuid': UUID('265bb071-3dd7-4a37-9806-3ade8ac29aac'),
 'number': 'TER-2024.00008',
 'company_id': UUID('4e3abcde-b827-469c-ab60-4e6a3030ee60'),
 'road_name': 'SPA-214/255',
 'road_id': 385,
 'km': 0.2,
 'end_km': 0.2,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7fe1ed6ef2e0>,
 'direction': '6',
 'lane': '99',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('f60fd4eb-432d-406b-828a-cd085e2025a1'),
 'firm_id': UUID('18301567-eadd-4fcb-81e7-0143f2ee2745'),
 'occurrence_type_id': UUID('f7858877-f5a5-4416-ba1a-39e5f320b30d'),
 'form_data': {'description': 'Retirar as pontas de ferro remanescentes da execução e tratar as áreas com argamassa polimérica'},
 'form_metadata': {},
 'executed_at': None,
 'created_at': datetime.datetime(2024, 2, 20, 15, 49, 27, 996485),
 'updated_at': None,
 'found_at': datetime.datetime(2024, 2, 20, 15, 49, 27, 996481),
 'due

In [93]:
result1 = bulk_create_with_history(jobs, Job, default_user=arteriscs)
result2 = bulk_create_with_history(terapia_objects, Reporting, default_user=arteriscs)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Job.start_date received a naive datetime (2025-06-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Job.end_date received a naive datetime (2025-07-13 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Job.start_date received a naive datetime (2024-07-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Job.end_date recei

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalJob.start_date received a naive datetime (2025-06-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalJob.end_date received a naive datetime (2025-07-13 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalJob.start_date received a naive datetime (2024-07-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:27.936383) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:27.970868) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:27.986685) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarnin

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:39.887772) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:40.006960) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:40.065801) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarnin

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:53.839310) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:53.873060) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-02-20 15:49:53.939671) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarnin

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:27.936383) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:27.970868) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:27.986685) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/_

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:40.337969) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:40.350836) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:40.361363) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/_

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:53.731214) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:53.797219) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-02-20 15:49:53.839310) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/_

In [4]:
from django.db.models import Q
companies=[]
for a in UserPermission.objects.filter(
    Q(permissions__has_key='contract')| 
    Q(permissions__has_key='Contract')
).order_by(
    'companies__name'
):
    for company in a.companies.all():
        if company.name not in companies:
            companies.append(company.name)

In [5]:
len(companies)

224

In [6]:
companies

['Llucena SINFRA',
 'Arteris Intervias',
 'OLD UHE-TESTE',
 'Demonstração - Energia',
 'POC-Projeto APR',
 'Teste Empreitera',
 'Garcia Monteiro',
 'ELO Teste',
 'ELO Rodovias Tietê',
 'ELO Ecopistas',
 'ELO Rodovias dos Tamoios',
 'ELO SP Vias Antiga (Não utilizada)',
 'ELO Rota das Bandeiras',
 'Arteris Planalto Sul - Conserva',
 'Arteris Litoral Sul - Conserva',
 'Arteris Fernão Dias - Conserva',
 'Arteris Fluminense - Conserva',
 'Arteris Régis Bittencourt - Conserva',
 'Demo Concessionárias',
 'Demo Teste',
 'Neovia - TESTE',
 'Sinalizavia (Municípios)',
 'Arteris Via Paulista - Faixa de Domínio',
 'Arteris Via Paulista - SISOBRAS/SISQUALI',
 'Arteris Via Paulista - SISOAE',
 'Eixo SP - Fx. Dom. | Qualidade | Topografia',
 'CAJ',
 'Arteris Via Paulista',
 'Ambiente Teste',
 'Construtora Castilho (Ferrovias)',
 'Arteris Litoral Sul',
 'Arteris Planalto Sul',
 'Arteris Régis Bittencourt',
 'UHE-PASSO FUNDO',
 'SETA',
 'Teste fluxo IMT',
 'Kartado | SIGACO',
 'Via Brasil SISQUALI',
 